In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
import json
from datasets import DatasetDict

In [ ]:

with open('Subtask_1_train.json', 'r', encoding = 'utf-8') as f:
    data = json.load(f)

context = []
conversation_data = ''
for conv in data:
    for utt in conv['conversation']:
        text = utt['text']
        conversation_data = conversation_data + ' ' + str(text)
    context.append(conversation_data)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
conversation_mapping = {}

for j in range(len(question)):
  for i in range(0, j):
    if i < len(question):
        conversation_mapping[tuple(question[i])] = context[i]


In [ ]:
temp_dict = DatasetDict({
              'context': context,
              'question': question
          })

In [ ]:
with open('Subtask_1_train.json', 'r', encoding = 'utf-8') as f:
    data = json.load(f)

context = []
question = []
answer = []
start_location = []
for conv in data:
    conversation_data = ''
    for utt in conv['conversation']:
        text = utt['text']
        conversation_data = conversation_data + ' ' + str(text)

    
    temp = []
    for emotion in conv['emotion-cause_pairs']:
        text = emotion[0][2:]
        temp.append(text)
        question.append(text)
    

    for i in range(len(temp)):
        context.append(conversation_data)


    for emotion in conv['emotion-cause_pairs']:
        text = emotion[1][2:]
        #print(text)
        test = conversation_data.find(text)
        #print(test)
        answers = {"text": [text], "answer_start": [test]}

        answer.append(answers)

In [ ]:
train_data = data_preprocess(train_data)

In [ ]:
val_data = data_preprocess(val_data)

In [ ]:
pip install transformers

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")

OSError: Can't load tokenizer for 'deepset/roberta-base-squad2'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'deepset/roberta-base-squad2' is the correct path to a directory containing all relevant files for a RobertaTokenizerFast tokenizer.

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_val = val_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/7548 [00:00<?, ? examples/s]

Map:   0%|          | 0/1816 [00:00<?, ? examples/s]

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [ ]:
from transformers import create_optimizer

batch_size = 16
num_epochs = 3
total_train_steps = (len(tokenized_train) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
)

In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForQuestionAnswering: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForQuestionAnswering for predictions without further training.


In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_train,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_val,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
model.compile(optimizer=optimizer)

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3)

Epoch 1/3
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


471/471 [==============================] - 3502s 7s/step - loss: 2.6266 - val_loss: 2.3346
Epoch 2/3
471/471 [==============================] - 3364s 7s/step - loss: 2.1890 - val_loss: 2.3066
Epoch 3/3
471/471 [==============================] - 3688s 8s/step - loss: 1.9953 - val_loss: 2.3283


In [ ]:
import pickle as pk
pk.dump(model, open('Roberta_model.pkl','wb'))

TESTING


In [ ]:
with open('val_data.json', 'r', encoding = 'utf-8') as f:
    data = json.load(f)

context = []
conversation_data = ''
for conv in data:
    for utt in conv['conversation']:
        text = utt['text']
        conversation_data = conversation_data + ' ' + str(text)
    context.append(conversation_data)

In [ ]:
context = context[2]
question = 'fear'

In [ ]:
context

' Hey ! So , what are you guys in the market for ? We have got uh , scarves , tulip post cards ... Check this out ? Huh ? Yeah . That is the stuff . What do you think ? Well , I do not have to buy that , " I am with stupid " T ... shirt anymore . Well , I like it . Here you go . All right , look , you are not really gonna buy that are you ? Do not you think you have embarrassed me enough for one day ? Oh , I embarrass you ? How can I answer that when I am pretending I do not know you ? Oh really ? Then how come no one here is wearing them ? They are all tourists . All right , look , if you insist on wearing that , in public , you know , you are gonna spend the rest of the afternoon all by yourself . Oh yeah ? If you are gonna make me choose between you and the hat , I choose the hat . Good choice . Thanks . Okay , wait . All right , that is it , okay , I am out of here . I am not going to be embarrassed anymore ! How about , you are moving ! ! Look ! This is ridiculous . We should be p

In [ ]:
inputs_1 = tokenizer(question, context, return_tensors="tf")

In [ ]:
outputs = model(**inputs_1)

In [ ]:
print(tf.math.top_k(outputs.start_logits, k = 3))
print(tf.math.top_k(outputs.end_logits, k = 3))


TopKV2(values=<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[4.270563 , 4.1364603, 3.7227194]], dtype=float32)>, indices=<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[51, 85, 95]])>)
TopKV2(values=<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[5.255456 , 4.6590776, 3.8241851]], dtype=float32)>, indices=<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[ 69, 107,  94]])>)


In [ ]:
answer_start_index = int(tf.math.argmax(outputs.start_logits, axis=-1)[0])
answer_start_index

51

In [ ]:
answer_end_index = int(tf.math.argmax(outputs.end_logits, axis=-1)[0])
answer_end_index

69

In [ ]:
predict_answer_tokens = inputs_1.input_ids[0, 51 : 69 + 1]
tokenizer.decode(predict_answer_tokens)

' I do not have to buy that, " I am with stupid " T... shirt anymore.'